
1. 1초 단위 전처리
2. 1% 단위로 데이터 행 추출 (100개 세트 or 이하)
3. 각 세트별 행 개수 구하기 (1%감소까지 걸린 초)
4. 초를 기준으로 빠름 보통 좋음으로 분류
5. 각 분류에서 높은 수치, 낮은 수치확인
6. x1,x2,x3.... 값 선정..

In [1]:
import scipy.io
import csv
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, chardet, glob
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [19]:
#TripA 타임 1초단위로 합하기
for i in range(32):
    globals()[i+1]=pd.read_csv('./TripA/TripA{}.csv'.format(i+1),sep=';')
    globals()[i+1] = globals()[i+1].iloc[::10,:]
    globals()[i+1].to_csv('./NewTripA/TripA{}.csv'.format(i+1),index=False)

In [ ]:
#TripB 타임 1초 단위로 합하기
for i in range(38):
    globals()[i+1]=pd.read_csv('./TripB/TripB{}.csv'.format(i+1),sep=';')
    globals()[i+1] = globals()[i+1].iloc[::10,:]
    globals()[i+1].to_csv('./NewTripB/TripB{}.csv'.format(i+1),index=False)

In [24]:
#TripA SoC 1퍼 단위로 각각 가져오기 및 저장
for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()[i].to_csv(f'./TripAsoc/TripA{j+1}_soc{i}.csv',index=False)
        startval += 1
        endval += 1

In [ ]:
#TripB SoC 1퍼 단위로 각각 가져오기 및 저장
for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()[i].to_csv(f'./TripBsoc/TripB{j+1}_soc{i}.csv',index=False)
        startval += 1
        endval += 1

In [30]:
globals()[1].shape[0]

3252

In [61]:
#TripA SoC 1%감소 데이터셋 명 : 행 개수로 딕셔너리 만들기
TripA_SoC_len = {}

for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['A_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            TripA_SoC_len ['TripA{}'.format(j+1)+' '+str(i)] = globals()[i].shape[0]
        startval += 1
        endval += 1


In [75]:
TripA_SoC_len

{'TripA1 81': 89,
 'TripA1 82': 164,
 'TripA1 83': 159,
 'TripA1 84': 195,
 'TripA1 85': 245,
 'TripA1 86': 157,
 'TripA2 66': 3,
 'TripA2 67': 93,
 'TripA2 68': 100,
 'TripA2 69': 134,
 'TripA2 70': 219,
 'TripA2 71': 176,
 'TripA2 72': 72,
 'TripA2 73': 187,
 'TripA2 74': 146,
 'TripA2 75': 21,
 'TripA2 76': 70,
 'TripA2 77': 57,
 'TripA2 78': 94,
 'TripA2 79': 34,
 'TripA2 80': 7,
 'TripA3 74': 82,
 'TripA3 75': 142,
 'TripA3 76': 105,
 'TripA3 77': 31,
 'TripA3 78': 29,
 'TripA3 79': 26,
 'TripA3 80': 19,
 'TripA3 81': 47,
 'TripA3 82': 131,
 'TripA3 83': 59,
 'TripA4 66': 107,
 'TripA4 67': 48,
 'TripA4 68': 23,
 'TripA4 69': 30,
 'TripA4 70': 41,
 'TripA4 71': 72,
 'TripA4 72': 21,
 'TripA4 73': 10,
 'TripA4 74': 19,
 'TripA4 75': 42,
 'TripA5 60': 250,
 'TripA5 61': 166,
 'TripA5 62': 328,
 'TripA5 63': 120,
 'TripA5 64': 158,
 'TripA5 65': 210,
 'TripA5 66': 135,
 'TripA6 63': 46,
 'TripA6 64': 328,
 'TripA6 65': 335,
 'TripA6 66': 54,
 'TripA6 67': 65,
 'TripA6 68': 93,
 'Trip

In [64]:
#TripB SoC 1% 감소 데이터셋 명 : 행 개수로 딕셔너리 만들기

TripB_SoC_len = {}

for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            TripB_SoC_len ['TripB{}'.format(j+1)+' '+str(i)] = globals()[i].shape[0]
        startval += 1
        endval += 1

In [65]:
TripB_SoC_len

{'TripB1 57': 171,
 'TripB1 58': 138,
 'TripB1 59': 58,
 'TripB1 60': 7,
 'TripB1 61': 70,
 'TripB1 62': 133,
 'TripB1 63': 21,
 'TripB1 64': 67,
 'TripB1 65': 147,
 'TripB1 66': 225,
 'TripB1 67': 165,
 'TripB1 68': 167,
 'TripB1 69': 137,
 'TripB1 70': 151,
 'TripB1 71': 140,
 'TripB1 72': 120,
 'TripB1 73': 21,
 'TripB1 74': 61,
 'TripB1 75': 115,
 'TripB1 76': 85,
 'TripB1 77': 60,
 'TripB1 78': 59,
 'TripB1 79': 193,
 'TripB1 80': 168,
 'TripB1 81': 150,
 'TripB1 82': 140,
 'TripB1 83': 73,
 'TripB1 84': 120,
 'TripB1 85': 67,
 'TripB1 86': 23,
 'TripB2 66': 160,
 'TripB2 67': 104,
 'TripB2 68': 16,
 'TripB2 69': 8,
 'TripB2 70': 86,
 'TripB2 71': 104,
 'TripB2 72': 66,
 'TripB2 73': 77,
 'TripB2 74': 218,
 'TripB2 75': 193,
 'TripB2 76': 151,
 'TripB2 77': 132,
 'TripB2 78': 76,
 'TripB2 79': 118,
 'TripB2 80': 101,
 'TripB2 81': 2,
 'TripB3 50': 118,
 'TripB3 51': 130,
 'TripB3 52': 10,
 'TripB3 53': 8,
 'TripB3 54': 64,
 'TripB3 55': 139,
 'TripB3 56': 30,
 'TripB3 57': 61,
 'T

In [70]:
# TripA SoC 1% 감소 행 개수 빠름 보통 좋음 으로 분류
TripA_SoC_len_classification = {}

for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            if globals()[i].shape[0] >= 150:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '좋음'
            elif globals()[i].shape[0] <50:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '빠름'
            else:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '보통'
        startval += 1
        endval += 1

In [72]:
TripA_SoC_len_classification

{'TripA1 81': '보통',
 'TripA1 82': '좋음',
 'TripA1 83': '좋음',
 'TripA1 84': '좋음',
 'TripA1 85': '좋음',
 'TripA1 86': '좋음',
 'TripA2 66': '빠름',
 'TripA2 67': '보통',
 'TripA2 68': '보통',
 'TripA2 69': '보통',
 'TripA2 70': '좋음',
 'TripA2 71': '좋음',
 'TripA2 72': '보통',
 'TripA2 73': '좋음',
 'TripA2 74': '보통',
 'TripA2 75': '빠름',
 'TripA2 76': '보통',
 'TripA2 77': '보통',
 'TripA2 78': '보통',
 'TripA2 79': '빠름',
 'TripA2 80': '빠름',
 'TripA3 74': '보통',
 'TripA3 75': '보통',
 'TripA3 76': '보통',
 'TripA3 77': '빠름',
 'TripA3 78': '빠름',
 'TripA3 79': '빠름',
 'TripA3 80': '빠름',
 'TripA3 81': '빠름',
 'TripA3 82': '보통',
 'TripA3 83': '보통',
 'TripA4 66': '보통',
 'TripA4 67': '빠름',
 'TripA4 68': '빠름',
 'TripA4 69': '빠름',
 'TripA4 70': '빠름',
 'TripA4 71': '보통',
 'TripA4 72': '빠름',
 'TripA4 73': '빠름',
 'TripA4 74': '빠름',
 'TripA4 75': '빠름',
 'TripA5 60': '좋음',
 'TripA5 61': '좋음',
 'TripA5 62': '좋음',
 'TripA5 63': '보통',
 'TripA5 64': '좋음',
 'TripA5 65': '좋음',
 'TripA5 66': '보통',
 'TripA6 63': '빠름',
 'TripA6 64': '좋음',


In [68]:
# TripB SoC 1% 감소 행 개수 빠름 보통 좋음 으로 분류
TripB_SoC_len_classification = {}

for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            if globals()[i].shape[0] >= 150:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '좋음'
            elif globals()[i].shape[0] <50:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '빠름'
            else:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '보통'
        startval += 1
        endval += 1

In [69]:
TripB_SoC_len_classification

{'TripB1 57': '좋음',
 'TripB1 58': '보통',
 'TripB1 59': '보통',
 'TripB1 60': '빠름',
 'TripB1 61': '보통',
 'TripB1 62': '보통',
 'TripB1 63': '빠름',
 'TripB1 64': '보통',
 'TripB1 65': '보통',
 'TripB1 66': '좋음',
 'TripB1 67': '좋음',
 'TripB1 68': '좋음',
 'TripB1 69': '보통',
 'TripB1 70': '좋음',
 'TripB1 71': '보통',
 'TripB1 72': '보통',
 'TripB1 73': '빠름',
 'TripB1 74': '보통',
 'TripB1 75': '보통',
 'TripB1 76': '보통',
 'TripB1 77': '보통',
 'TripB1 78': '보통',
 'TripB1 79': '좋음',
 'TripB1 80': '좋음',
 'TripB1 81': '좋음',
 'TripB1 82': '보통',
 'TripB1 83': '보통',
 'TripB1 84': '보통',
 'TripB1 85': '보통',
 'TripB1 86': '빠름',
 'TripB2 66': '좋음',
 'TripB2 67': '보통',
 'TripB2 68': '빠름',
 'TripB2 69': '빠름',
 'TripB2 70': '보통',
 'TripB2 71': '보통',
 'TripB2 72': '보통',
 'TripB2 73': '보통',
 'TripB2 74': '좋음',
 'TripB2 75': '좋음',
 'TripB2 76': '좋음',
 'TripB2 77': '보통',
 'TripB2 78': '보통',
 'TripB2 79': '보통',
 'TripB2 80': '보통',
 'TripB2 81': '빠름',
 'TripB3 50': '보통',
 'TripB3 51': '보통',
 'TripB3 52': '빠름',
 'TripB3 53': '빠름',


In [ ]:
#5번
#세트별로 모든 파라미터의 평균값을 내고
#좋은 세트일 때의 각 칼럼들의 평균 중 높은 것들이 뭐인지 보고
#나쁜 세트일 떄의 각 칼럼들의 평균 중 높은 것들이 뭐인지 보고


In [ ]:
#A세트 행 계수들 확인
for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    globals()['A_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()['A_len_{}'.format(j+1)][i] = globals()[i].shape[0]
        startval += 1
        endval += 1

#B세트 행 계수들 확인
for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()['B_len_{}'.format(j+1)][i] = globals()[i].shape[0]
        startval += 1
        endval += 1